In [128]:
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
import itertools

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import torchsummary
import copy

import random
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
# Colab specific data loading

from google.colab import drive
drive.mount('/gdrive', force_remount=True)


Mounted at /gdrive


In [4]:

# In Google Drive, create a shortcut link of the shared Project folder so that it appears in your root Google Drive directory
# i.e., Right Click Project folder: "Add Shortcut to Drive"

%cd '/gdrive/MyDrive/Project/'


/gdrive/.shortcut-targets-by-id/1ONXXjIHVMg4MHpAdUIdbB4Fr9W33JPYU/Project


In [5]:
# # Data should be in a dir named 'data/'
data = np.load('data/RESISC45_images.npy')
labels = np.load('data/RESISC45_classes.npy')
classes = np.load('data/RESISC45_class_names.npy')


In [ ]:

print('Training data shape: ', data.shape)
print('Testing data shape: ', labels.shape)
print('Num Classes', classes.shape)


In [6]:

class addGaussianNoise(object):
    # Transform to add gaussian noise since PyTorch did not have one (that I know of).
    def __init__(self, mean=0.0, std=1.0, p=0.5):
        self.mean = mean
        self.std = std
        self.p = p
      
    def __call__(self, img):
        if torch.rand(1).item() < self.p:
            return img + torch.randn(img.size()) * self.std + self.mean
        return img
        
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1}, p={2})'.format(self.mean, self.std, self.p)



In [123]:

def applyAugs(img_batch, current_task):
    # conduct transform operations for task and return transformed batch of images

    transform_list = [transforms.RandomHorizontalFlip(p=0.99),
                      transforms.RandomVerticalFlip(p=0.99),
                      transforms.RandomRotation(359.0, fill=0.5),
                      transforms.RandomPerspective(distortion_scale=0.1, p=0.99, fill=0.5),
                      transforms.RandomResizedCrop(256,
                                                   scale=(0.5, 1.0),
                                                   ratio=(1.0, 1.0),
                                                   interpolation=transforms.InterpolationMode.BILINEAR),
                      addGaussianNoise(std=0.1, p=0.99),
                      # transforms.ColorJitter(saturation=4.0, hue=0.01),
                      transforms.ColorJitter(brightness=0.5, contrast=0.9)
                      # ,transforms.GaussianBlur(9, sigma=(0.01, 2.0))
                      ]
    
    task_transforms = [transform_list[i] for i,x in enumerate(current_task) if x==1]

    transform = torchvision.transforms.Compose(task_transforms)
    img_batch = transform(img_batch)

    return img_batch



In [9]:

# Create full-factorial combination of the augmentations. 
numAugs = 7
augTasks = list(itertools.product([0, 1], repeat=numAugs))


In [159]:

# prepare a minibatch of images
batch_size = 256

d = data[0:batch_size]
d = d / 255.0
d = np.moveaxis(d, 3, 1)
d = torch.as_tensor(d)


In [10]:

# select random task (for testing)
# in actual implementation we would iterate through the task list
current_task = augTasks[random.randint(0, 2**numAugs)]


In [ ]:

# # check timing 

# # transform minibatch of images
# t0 = time.time()
# transformed_batch = applyAugs(d, current_task)
# t1 = time.time()
# print("Time to transform batch of size {:d}: {:0f}".format(len(d), t1-t0))

# print("estimated total time for all images all tasks: {:.1f} hours".format((31500/batch_size * (t1-t0)) * (2**numAugs)/60/60/2))


In [ ]:

# # check timings for individual transforms

# for i in range(0, numAugs):
#   current_task = [0] * numAugs
#   current_task[i] = 1
#   t0 = time.time()
#   transformed_batch = applyAugs(d, current_task)
#   print("Aug:", i, "{:0f}".format(time.time()-t0))


In [ ]:

# # transform all images

# # test with all augs on
# current_task = [1] * numAugs

# t0 = time.time()
# for i in range(0, data.shape[0], batch_size):
#   print(i, time.time() - t0)

#   d = data[i:i+batch_size]
#   d = d / 255.0
#   d = np.moveaxis(d, 3, 1)
#   d = torch.as_tensor(d)

#   transformed_batch = applyAugs(d, current_task)

# print("Time to transform entire dataset {:d}: {:0f}".format(data.shape[0], time.time()-t0))



In [ ]:

# # plot sample of results
# from mpl_toolkits.axes_grid1 import ImageGrid

# transformed_batch = transformed_batch.numpy()
# transformed_batch = np.moveaxis(transformed_batch, 1, 3)

# fig = plt.figure(figsize=(15., 15.))
# grid = ImageGrid(fig, 111,  
#                  nrows_ncols=(4, 4),  
#                  axes_pad=0.1)

# for ax, im in zip(grid, transformed_batch[0:16]):
#     # Iterating over the grid returns the Axes.
#     ax.imshow(im)

# plt.show()

# del transformed_batch

